# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

with zipfile.ZipFile('Orders.zip', 'r') as zip_ref:
    zip_ref.extractall()

orders = pd.read_csv('Orders.csv')
orders.head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [12]:
# your code here

customerlabels = ['Cheap', 'Average', 'Preferred', 'VIP']
quantiles = [.25,.50,.75,.95, 1]
binning = pd.qcut(orders["amount_spent"],quantiles, labels= customerlabels)
orders['customer_type'] = binning
orders[['amount_spent','customer_type']].head(10)


,amount_spent,customer_type
0,15.30,Average
1,20.34,Preferred
2,22.00,Preferred
3,20.34,Preferred
4,20.34,Preferred
5,15.30,Average
6,25.50,Preferred
7,11.10,Cheap
8,11.10,Cheap
9,54.08,Preferred


In [13]:
orders.groupby(["customer_type"])["amount_spent"].sum()

customer_type
Cheap         797564.50
Average      1747446.50
Preferred    2245195.30
VIP          3887283.52
Name: amount_spent, dtype: float64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [14]:
# your code here

VIP_countries = orders.loc[orders['customer_type'] == 'VIP', ['customer_type','Country']].groupby("Country").count()
VIP_countries = VIP_countries.rename(columns ={'customer_type' : 'VIP_customers'})
VIP_countries.sort_values(by='VIP_customers').tail(1)

,VIP_customers
Country,
United Kingdom,14739


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [16]:
# your code here

preferred_countries = orders.loc[orders['customer_type'] == 'Preferred', ['customer_type','Country']].groupby("Country").count()
preferred_countries = preferred_countries.rename(columns ={'customer_type' : 'preferred_customers'})
preferred_countries.head(10)

,preferred_customers
Country,
Australia,272
Austria,91
Bahrain,9
Belgium,542
Brazil,12
Canada,30
Channel Islands,268
Cyprus,170
Czech Republic,19


In [17]:
total = pd.concat([VIP_countries, preferred_countries], axis=1, sort=False)
total.head(10)

,VIP_customers,preferred_customers
Australia,571.0,272
Austria,27.0,91
Bahrain,2.0,9
Belgium,39.0,542
Brazil,3.0,12
Canada,3.0,30
Channel Islands,31.0,268
Cyprus,24.0,170
Czech Republic,1.0,19
Denmark,62.0,155


In [18]:
total['Total'] = total.sum(axis=1)

In [19]:
total.sort_values(by='Total').tail(1)

,VIP_customers,preferred_customers,Total
United Kingdom,14739.0,58205,72944.0
